# Alternating Least Square (ALS)

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from interaction_table import orders_weigher, InteractionTable
from h3_index import H3Index

In [2]:
#!pip install fastparquet
h3index = H3Index('../data/h3_to_chains.pkl')

In [3]:
def get_clicks():
    path = '../data/clicks/click.parquet'
    return pd.read_parquet(path)

def get_orders():
    path = '../data/orders/orders.parquet'
    df = pd.read_parquet(path)
    df = df.rename(columns={"customer_id": "user_id"})
    return df

In [19]:
interactions = InteractionTable(None, get_orders,
                                None, orders_weigher,
                                alpha=0)

Orders df loaded: size=14862643,  uniq_users=3973431,  uniq_chains=23887
Orders weighter: use user avg orders per chain as weight
            user_id      chain_id        weight
count  8.499794e+06  8.499794e+06  8.499794e+06
mean   3.989632e+07  3.527085e+04  1.748589e+00
std    2.128882e+07  1.553233e+04  5.205855e+01
min    0.000000e+00  9.000000e+00  1.000000e+00
25%    2.897376e+07  2.872000e+04  1.000000e+00
50%    4.474124e+07  3.204900e+04  1.000000e+00
75%    5.502611e+07  4.660900e+04  2.000000e+00
max    7.213902e+07  7.332400e+04  1.444470e+05
Orders df weighted: size=8499794, uniq_users=3973431, uniq_chains=23887


In [21]:
test = interactions.interaction_df[['user_id', 'weight']]
test = test.groupby('user_id').sum()
test = test.reset_index()[['user_id', 'weight']]
user_with_few_interactions = set(test[test['weight'] <= 5].user_id.unique())
interactions_small = interactions.interaction_df.query('user_id not in @user_with_few_interactions')
interactions_small.to_parquet('../data/interaction_by_orders.parquet')
len(interactions_small)

3463671

In [25]:
orders = get_orders()[['user_id', 'chain_id']]
orders = orders.query('user_id not in @user_with_few_interactions')
orders['weight'] = 1 # dummy
orders.to_parquet('../data/ncf_orders.parquet')
len(orders)

8480907

In [7]:
val_df = pd.read_pickle('../data/test_VALID.pkl')
val_df = val_df[['customer_id', 'h3', 'chain_id']]
val_df = val_df.rename(columns={"customer_id": "user_id"})
val_df.user_id = val_df.user_id.astype(int)
print(len(val_df))
val_df = val_df.query('h3 in @h3index.valid')
print(len(val_df))
val_df = val_df.query('user_id in @interactions.user_index')
print(len(val_df))
val_df = val_df.query('chain_id in @interactions.chain_index')
print(len(val_df))
val_df = pd.pivot_table(val_df,
                        values=['chain_id'],
                        index=['user_id', 'h3'],
                        aggfunc={'chain_id': set})
val_df = val_df.reset_index()
val_df.head()

2300001
2293762
45860
45153


,user_id,h3,chain_id
0,392,891181b655bffff,"{41874, 28795, 45789}"
1,600,8911aa44d53ffff,{2046}
2,1370,8911aa7a523ffff,{27199}
3,5138,8911aa6aeb3ffff,{10807}
4,6082,8911aa08e73ffff,{48274}


### Если h3 пользователя неизвестен, то можно брать следующий в иерархии h3 (более крупный)

In [27]:
def predict(model, user_id, h3, thr=0.9, top_k=10, filter_already_liked_items=True):
    user_index = interactions.user_index[user_id]
    valid_chains = h3index.h3_to_chains[h3]
    filter_items = [v for k, v in interactions.chain_index.items() if k not in valid_chains]
    top = model.recommend(user_index,
                          interactions.sparse_interaction_matrix.T,
                          N=top_k,
                          filter_already_liked_items=filter_already_liked_items,
                          filter_items=filter_items)
    top = [interactions.r_chain_index[x] for x, score in top if score > thr]
    return top

def old_items(user_id):
    return set(interactions.interaction_df[interactions.interaction_df['user_id'] == user_id]['chain_id'].unique())

In [28]:
def metric(y_true, y_pred, y_old, at1=10, at2=30, average=True):
    """
    new_prec@10 + new_prec@30 + 1/2 *(prec_@10 + prec@30)
    """
    scores_new = []
    scores_all = []
    scores_total = []
    for t, p, o in zip(y_true, y_pred, y_old):
        t = list(t)
        p = list(p)
        o = o if isinstance(o, (set, list)) else []
        
        prec1 = len(set(t[:at1]) & set(p[:at1])) / at1
        prec2 = len(set(t[:at2]) & set(p[:at2])) / at2
        new_prec1 = len((set(p[:at1]) - set(o)) & set(t[:at1])) / at1
        new_prec2 = len((set(p[:at2]) - set(o)) & set(t[:at2])) / at2

        scores_total.append(new_prec1 + new_prec2 + 0.5 * (prec1 + prec2))
        scores_new.append(new_prec1 + new_prec2)
        scores_all.append(prec1 + prec2)

    return (np.mean(scores_total) if average else scores_total,
            np.mean(scores_new) if average else scores_new,
            np.mean(scores_all) if average else scores_all)

In [29]:
# !pip install implicit
import implicit

def hyper_params(val_df, factors=60, thr=0.7, top_k=30, filter_liked=True):
    print('factors: ', factors, ', thr: ', thr, ', top_k: ', top_k, ', filter_liked: ', filter_liked)
    model = implicit.als.AlternatingLeastSquares(factors=factors)
    model.fit(interactions.sparse_interaction_matrix)
    val = val_df
    val['pred_chains'] = val.apply(lambda x: predict(model, x.user_id, x.h3, thr, top_k, filter_liked), axis=1)
    val['old_chains'] = val.apply(lambda x: old_items(x.user_id), axis=1)
    scores = metric(val['chain_id'], val['pred_chains'], val['old_chains'])
    print('total, new, all = ', scores)
    print()

In [30]:
hyper_params(val_df, factors=60, thr=0.7, top_k=30, filter_liked=True)

factors:  60 , thr:  0.7 , top_k:  30 , filter_liked:  True


  0%|          | 0/15 [00:00<?, ?it/s]

total, new, all =  (0.026074051131157485, 0.00025294565540026067, 0.051642210951514445)



factors:  60 , thr:  0.7 , top_k:  30 , filter_liked:  True

total, new, all =  (0.02605082142811052, 0.00023745918670228555, 0.05162672448281647)

In [ ]:
for factors in [30, 40, 50, 60, 70]:
    for thr in [0.7, 0.75, 0.8, 0.85, 0.9]:
        for top_k in [5, 10, 20, 30]:
            for filter_liked in [True, False]:
               hyper_params(val_df, factors, thr, top_k, filter_liked) 